In [171]:
# Import the pandas package, then use the "read_csv" function to read
# the labeled training data
import sklearn
import pandas as pd
import numpy as np
from collections import Counter
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy.linalg as LA
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")



def most_common(list):
    data = Counter(list)
    if data.most_common(1)[0][0] ==1:
        return "+1"
    else: 
        return "-1"
def majority_vote(indicies, sentiment):
    sentimentlist = []
    for index in indicies:
        sentimentlist.append(sentiment[index])
    return most_common(sentimentlist)

def getNeighbors(testinstance, train_set, k):
    cosine_similarities = cosine_similarity(testinstance, train_set).flatten()
    related_docs_indices = cosine_similarities.argsort()[:-(k+2):-1]
    x =cosine_similarities[related_docs_indices]
    x = list(x)
    neighbors = x[1:]
    return neighbors

def getResponse(testinstance, train_set, k):
    cosine_similarities = cosine_similarity(testinstance, train_set).flatten()
    related_docs_indices = cosine_similarities.argsort()[:-(k+2):-1]
    related_docs_indices = related_docs_indices[1:]
    return related_docs_indices

#import train data into pandas
train = pd.read_csv("train.data", header = None, names = ['sentiment', 'review'],delimiter="\t")

#import test data into pandas dataframe
import os
file = open(os.path.join( os.path.dirname('_file_'), 'test.data'), 'r')
test1 = file.readlines()
test1 =list(test1)
test = pd.DataFrame(test1)

#test = pd.read_csv('test.data', header = None, names = ['review'], delimiter= '\t')

#Positve/Negative senntiment, which is stored as an array
sentiment = train['sentiment']
sentiment = np.asarray(sentiment)

#Converted Pandas series format to list containing the Training reviews
train_set = []
for index, row in train.iterrows():
    review  = str(train["review"][index])
    train_set.append(review)

#Converted Pandas series format to list containing the test reviews
test_set = []
for index, row in test.iterrows():
    review = str(train["review"][index])
    test_set.append(review)


[nltk_data] Downloading package stopwords to /home/neil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [172]:
#Extract Features into a sparse matrix from training data using vectorizer
tfidf_vectorizer = TfidfVectorizer( analyzer = 'word',
                                    stop_words='english',
                                    max_features = 3000
                                  )
tfidf_train = tfidf_vectorizer.fit_transform(train_set)
print tfidf_train.shape

(18506, 3000)


In [173]:
tfidf_test = tfidf_vectorizer.transform(test_set)
print tfidf_test.shape

(18506, 3000)


In [174]:
#make sentiment predictions over entire test set
predictions = []

for documents in range(0,18506):
    indicies = getResponse(tfidf_test[documents], tfidf_train, 19)
    predictions.append(majority_vote(indicies, sentiment))


In [176]:
f = open('predictions.dat', 'w')
f.write("\n".join(map(lambda x: str(x), predictions)))
f.close()